In [1]:
import numpy as np
import scipy.stats as sts
import matplotlib.pyplot as plt
import heapq

In [2]:
class Event:
    '''
    Store the properties of one event in the Schedule class defined below. Each
    event has a time at which it needs to run, a function to call when running
    the event, along with the arguments and keyword arguments to pass to that
    function.
    '''
    def __init__(self, timestamp, function, *args, **kwargs):
        self.timestamp = timestamp
        self.function = function
        self.args = args
        self.kwargs = kwargs

    def __lt__(self, other):
        '''
        This overloads the less-than operator in Python. We need it so the
        priority queue knows how to compare two events. We want events with
        earlier (smaller) times to go first.
        '''
        return self.timestamp < other.timestamp

    def run(self, schedule):
        '''
        Run an event by calling the function with its arguments and keyword
        arguments. The first argument to any event function is always the
        schedule in which events are being tracked. The schedule object can be
        used to add new events to the priority queue.
        '''
        self.function(schedule, *self.args, **self.kwargs)


class Schedule:
    '''
    Implement an event schedule using a priority queue. You can add events and
    run the next event.
    
    The `now` attribute contains the time at which the last event was run.
    '''
    
    def __init__(self, busSystem):
        self.now = 0  # Keep track of the current simulation time
        self.priority_queue = []  # The priority queue of events to run
        self.busSystem = busSystem
        
    def add_event_at(self, timestamp, function, *args, **kwargs):
        # Add an event to the schedule at a particular point in time.
        heapq.heappush(
            self.priority_queue,
            Event(timestamp, function, *args, **kwargs))
    
    def add_event_after(self, interval, function, *args, **kwargs):
        # Add an event to the schedule after a specified time interval.
        self.add_event_at(self.now + interval, function, *args, **kwargs)
    
    def next_event_time(self):
        # Return the time of the next event. The `now` attribute of this class
        # contain the time of the last event that was run.
        return self.priority_queue[0].timestamp

    def run_next_event(self):
        # Get the next event from the priority queue and run it.
        event = heapq.heappop(self.priority_queue)
        self.now = event.timestamp
        event.run(self)
        
    def __repr__(self):
        return (
            f'Schedule() at time {self.now} ' +
            f'with {len(self.priority_queue)} events in the queue')
    
    def print_events(self):
        # Print out diagnostic information about the events in the schedule.
        print(repr(self))
        for event in sorted(self.priority_queue):
            print(f'   {event.timestamp}: {event.function.__name__}')

In [3]:
class Queue:
    '''
    This class tracks the number of people in the queue and the number of
    people being served. It also schedules when a customer starts being served and
    stops being served.
    '''
    def __init__(self, waiting_num, served_num):
        self.waiting_num = 0
        self.served_num = 0
    

In [4]:
class BusSystem:
    '''
    This class contains the queues (in this case only one 
    but you’ll need to make more than one for your project). 
    It also schedules the arrival of customers at the queue(s).
    '''
    
    def __init__(self, queue, service_time, arrival_distribution):
        
        self.queue = queue
        self.service_time = service_time
        self.arrival_distribution = arrival_distribution
        
        #for plotting the dats from a dictionary
        self.customers_per_time ={"time":[], "count_queue":[]}
        
    def schedule_arrival(self, schedule):
        # calculating the next arrival time
        next_arrival = schedule.now + self.arrival_distribution.rvs()
        # making an event of the arrival
        customer_arrives = Event(next_arrival, customer_serving)
        # add an event
        schedule.add_event_at(customer_arrives.timestamp, customer_arrives.function)
        # increase the queue
        self.queue.count_queue += 1
        schedule.now = customer_arrives.timestamp

In [5]:
def customer_serving(schedule):
    
    serve_rate = schedule.busSystem.service_time
    
    schedule.now_ = schedule.now
    while schedule.now < schedule.now_ + serve_rate:
        schedule.busSystem.schedule_arrival(schedule)
    
    schedule.busSystem.queue.count_queue -= 1
    schedule.busSystem.customers_per_time["time"].append(schedule.now)
    schedule.busSystem.customers_per_time["count_queue"].append(schedule.busSystem.queue.count_queue)

In [6]:
arrival_rate = 1
service_rate = 5
run_until = 20

def run_simulation(arrival_rate, service_rate, run_until):
    
    arrival_dist = sts.expon(scale=1/arrival_rate)
    
    queue = Queue(0,0)
    
    busSystem = BusSystem(queue, arrival_dist, service_rate)
    
    schedule = Schedule(busSystem)
    busSystem.schedule_arrival(schedule)
    
    
    while schedule.next_arrival() <= run_until:
        schedule.run_next_event()
    
    
    plt.plot(busSystem.customers_per_time["time"], busSystem.customers_per_time["count_queue"] )
    plt.legend()
        
run_simulation(arrival_rate, service_rate, run_until)

AttributeError: 'int' object has no attribute 'rvs'

### Reflection

My code has some problems that I am not sure how to solve it. According to to the study guide, I created 2 more classes: Class Queue and Class BusSystem to keep track of the system stutus. I also created one function to run the simulation. My objective for this pre-class is understanding the sample code and how to create a model for simulation for the next assignment. 